In [2]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

# Load Dataset
df = pd.read_csv("C:/Users/amish/OneDrive/Desktop/Kaggle Files/ratings_Electronics.csv", names=["reviewerID", "asin", "rating", "timestamp"], dtype={"reviewerID": str, "asin": str})
df.drop(columns=["timestamp"], inplace=True)

# Encode Users & Items
df["user_idx"] = df["reviewerID"].astype('category').cat.codes
df["item_idx"] = df["asin"].astype('category').cat.codes

num_users = df["user_idx"].nunique()
num_items = df["item_idx"].nunique()

# Create Sparse Interaction Matrix
interaction_matrix = csr_matrix((df['rating'], (df['user_idx'], df['item_idx'])), shape=(num_users, num_items))

# Train k-NN Model (Item-Based Collaborative Filtering)
k = 10  # Neighbors to consider
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=k, n_jobs=-1)
knn.fit(interaction_matrix.T)  # Transpose for item-item similarity

# Recommendation Function
def recommend_items(user_id, top_n=5):
    if user_id not in df['user_idx'].values:
        return "❌ User not found. Try another user."
    
    user_interactions = interaction_matrix[user_id].toarray().flatten()
    watched_items = np.where(user_interactions > 0)[0]
    
    if len(watched_items) == 0:
        return "📌 No interactions found for this user. Showing popular items instead."
    
    scores = {}
    for item in watched_items:
        distances, indices = knn.kneighbors(interaction_matrix.T[item], n_neighbors=k)
        for i in range(1, k):  # Skip first because it's the item itself
            neighbor = indices[0][i]
            if neighbor not in watched_items:
                scores[neighbor] = scores.get(neighbor, 0) + (1 - distances[0][i])
    
    recommendations = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:top_n]
    return [(df.loc[df['item_idx'] == idx, 'asin'].values[0], score) for idx, score in recommendations]

# Example Usage
user_id = 0  # Change this to test different users
print(recommend_items(user_id))


[('B00005T3XD', np.float64(0.12700012700019048)), ('B001AEWKXK', np.float64(0.12700012700019048)), ('B000VB6S9U', np.float64(0.12700012700019048)), ('B0026Q0EJS', np.float64(0.12320822072673954)), ('B00006OXGH', np.float64(0.10160010160015243))]


In [3]:
# Display Recommendations in a more structured way
def show_recommendations(user_id, top_n=5):
    recommended_items = recommend_items(user_id, top_n)
    
    if not recommended_items:
        print(f"❌ No personalized recommendations available for User {user_id}. Showing popular items instead.")
    else:
        print(f"\n🎯 Personalized Recommendations for User {user_id}:")
        for idx, (item, _) in enumerate(recommended_items, start=1):
            if idx == 1:
                print(f"🔥 {idx}. {item} - This is a **highly relevant** recommendation for you!")
            elif idx <= 3:
                print(f"✅ {idx}. {item} - This should be a **good match** for your interests.")
            else:
                print(f"👍 {idx}. {item} - You might find this interesting.")

# Example Usage
show_recommendations(user_id=0, top_n=5)  # Replace with actual user_id


🎯 Personalized Recommendations for User 0:
🔥 1. B00005T3XD - This is a **highly relevant** recommendation for you!
✅ 2. B001AEWKXK - This should be a **good match** for your interests.
✅ 3. B000VB6S9U - This should be a **good match** for your interests.
👍 4. B0026Q0EJS - You might find this interesting.
👍 5. B00006OXGH - You might find this interesting.
